In [34]:
import sys, glob, os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time


from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table, hstack, join, vstack, unique


In [52]:
flats = glob.glob("C:/Users/Vicen/OneDrive/Escritorio/astro obs/fotometria/imagenes/ngc869/Jhonson B/30s/flats/*fit")
print('Flats available: ', len(flats), '\n')

flat_darks=glob.glob("C:/Users/Vicen/OneDrive/Escritorio/astro obs/fotometria/imagenes/ngc869/Jhonson B/30s/darks/*fit")
print('Darks from flats exp time available: ', len(flat_darks), '\n')

darks=glob.glob("C:/Users/Vicen/OneDrive/Escritorio/astro obs/fotometria/imagenes/ngc869/Jhonson B/30s/darks/*fit")
print('Darks available: ', len(darks), '\n')

images=glob.glob("C:/Users/Vicen/OneDrive/Escritorio/astro obs/fotometria/imagenes/ngc869/Jhonson B/30s/lights/*fit")
print('Lights availible: ', len(images), '\n')

Corregides="C:/Users/Vicen/OneDrive/Escritorio/astro obs/fotometria/imagenes/ngc869/Jhonson B/30s/Corregidas"

Object='NGC869'

Flats available:  9 

Darks from flats exp time available:  4 

Darks available:  4 

Lights availible:  4 



## Master dark

In [36]:
#Darks amb temps d'exposició de imatges
fit=fits.open(darks[0])

arr=(fit[0].data)
for i in range(1,len(darks)):
    fit=fits.open(darks[i])
    new_arr=fit[0].data
    arr+=new_arr
master_dark=arr/len(darks)

hdr=fit[0].header
hdr['File_type']='Master_Dark'
MasterDark_fits = fits.PrimaryHDU(master_dark, header=hdr)


In [37]:
MasterDark_fits.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 4096                                                  
NAXIS2  =                 4096                                                  
DATE-OBS= '2023-11-23T17:49:42' /YYYY-MM-DDThh:mm:ss observation start, UT      
EXPTIME =   30.000000000000000 /Exposure time in seconds                        
EXPOSURE=   30.000000000000000 /Exposure time in seconds                        
SET-TEMP=  -15.000000000000000 /CCD temperature setpoint in C                   
CCD-TEMP=  -14.956620216369629 /CCD temperature at start of exposure in C       
XPIXSZ  =   9.0000000000000000 /Pixel Width in microns (after binning)          
YPIXSZ  =   9.0000000000000000 /Pixel Height in microns (after binning)         
XBINNING=                   

In [38]:
MasterDark_fits.data

array([[542.  , 542.25, 539.  , ..., 545.25, 542.75, 545.75],
       [555.75, 538.75, 538.5 , ..., 536.75, 539.75, 543.25],
       [549.75, 538.75, 538.5 , ..., 531.75, 536.75, 539.75],
       ...,
       [554.25, 545.75, 544.5 , ..., 537.  , 538.  , 550.5 ],
       [554.25, 540.  , 538.75, ..., 533.25, 535.5 , 549.25],
       [557.75, 538.75, 547.  , ..., 539.75, 540.5 , 541.75]])

In [39]:
#Dark amb temps d'exposició de flats

fit=fits.open(flat_darks[0])
arr=(fit[0].data)
for i in range(1,len(flat_darks)):
    fit=fits.open(flat_darks[i])
    new_arr=fit[0].data
    arr+=new_arr
masterdark_de_flats=arr/len(flat_darks)

hdr=fit[0].header
hdr['File_type']='Master_Dark_t_exp_flats'
MasterDark_de_flats_fits = fits.PrimaryHDU(masterdark_de_flats, header=hdr)

In [40]:
MasterDark_de_flats_fits.data

array([[542.  , 542.25, 539.  , ..., 545.25, 542.75, 545.75],
       [555.75, 538.75, 538.5 , ..., 536.75, 539.75, 543.25],
       [549.75, 538.75, 538.5 , ..., 531.75, 536.75, 539.75],
       ...,
       [554.25, 545.75, 544.5 , ..., 537.  , 538.  , 550.5 ],
       [554.25, 540.  , 538.75, ..., 533.25, 535.5 , 549.25],
       [557.75, 538.75, 547.  , ..., 539.75, 540.5 , 541.75]])

## Correcció de flats

In [41]:

fit=fits.open(flats[0])
arr=np.array((fit[0].data), dtype=np.float64)
for i in range(1,len(flats)):
    fit=fits.open(flats[i])
    new_arr=fit[0].data - masterdark_de_flats
    arr+=new_arr
    
masterflat=arr/len(flats)

hdr=fit[0].header
hdr['File_type']='Master_Flat'
MasterFlat_fits = fits.PrimaryHDU(masterflat, header=hdr)

In [42]:
print(MasterFlat_fits.data)

[[14350.88888889 14095.55555556 14209.33333333 ... 13727.
  13994.         14158.88888889]
 [14222.88888889 14316.22222222 13960.         ... 13837.
  13787.         13883.11111111]
 [14389.88888889 14013.66666667 13900.11111111 ... 13826.44444444
  13774.22222222 13868.66666667]
 ...
 [14294.11111111 14116.77777778 14424.55555556 ... 14242.77777778
  14083.55555556 14293.44444444]
 [14108.55555556 14360.22222222 14114.55555556 ... 14275.22222222
  14444.33333333 14425.44444444]
 [14700.77777778 14165.66666667 14056.         ... 14154.33333333
  14330.55555556 14173.77777778]]


## Correcció de lights

In [43]:
fit_image=fits.open(images[0])
fit_image.info()

Filename: C:/Users/arnau/OneDrive/Escritorio/astro obs/fotometria/imagenes/ngc869/Jhonson B/30s/lights\NGC869-001_B_30s.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      41   (4096, 4096)   int16 (rescales to uint16)   


In [65]:
media_flat=np.median(MasterFlat_fits.data)

counter = 0
for im in images:
    fit_image=fits.open(im)
    fit_corr=(fit_image[0].data-MasterDark_fits.data)/(MasterFlat_fits.data)*media_flat
    
    hdr=fit_image[0].header
    hdr['File_type']='Corrected_image'
    Corr_image_fits = fits.PrimaryHDU(fit_corr, header=hdr)
    
    Corr_image_fits.writeto(Corregides+'/'+Object+'_'+hdr['Filter']+'_'+str(hdr['EXPtime'])+'s_'+str(counter)+"_corr.fits")
    counter += 1

In [64]:
print(im)

C:/Users/arnau/OneDrive/Escritorio/astro obs/fotometria/imagenes/ngc869/Jhonson B/30s/lights\NGC869-001_B_30s.fit


In [62]:
path='C:/Users/arnau/OneDrive/Escritorio/astro obs/fotometria/imagenes/ngc869/Jhonson B/30s/lights'
print(Corregides+'/'+Object+'_'+hdr['Filter']+'_'+str(hdr['EXPtime'])+'s_'+str(counter)+"_corr.fits")

C:/Users/arnau/OneDrive/Escritorio/astro obs/fotometria/imagenes/ngc869/Jhonson B/30s/Corregidas/NGC869_Johnson B_1.0s_0_corr.fits
